# Introduction to NLP fundamentals in TensorFlow

NLP has the goal of deriving information our of natural language (could be text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e14c7b80-c091-4014-ffa1-d9dd9d82ac17)


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-04-16 08:44:56--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-04-16 08:44:56 (90.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster).

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-04-16 08:45:02--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.196.128, 173.194.197.128, 64.233.191.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.196.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2023-04-16 08:45:02 (91.3 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Let's visualize some random training samples
import random
random_index = random.randint(0, len(train_df) - 5) # create random indexes

for row in train_df_shuffled[["text", "target"]][random_index:random_index + 5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "not real disaster")
  print(f"Text:\n{text}\n")
  print()

Target: 1 (real disaster)
Text:
US wont upgrade its infrastructure? http://t.co/NGEHhG9YGa' it a bad situation and its going to get ugly very quickly #USA #sustainability


Target: 0 not real disaster
Text:
#TBT to that time my best friend and I panicked at the disco. https://t.co/htpqvoHtUd


Target: 0 not real disaster
Text:
SCREAMING IN 22 DIFFERENT LANGUAGES http://t.co/rDfaAKKbNJ


Target: 0 not real disaster
Text:
*to Luka* They should all die! All of them! Everything annihilated! - Alois Trancy


Target: 1 (real disaster)
Text:
Medieval airplane hijacker testa: earnings the distinction divers: HtaRvrGLY




### Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                  train_df_shuffled["target"].to_numpy(), 
                                                  test_size=0.1,
                                                  random_state=42)

In [11]:
# Check the lenghts
len(X_train), len(X_val), len(y_train), len(y_val)

(6851, 762, 6851, 762)

In [12]:
# Check the first 10 samples
X_train[:10], y_train[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character)
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [13]:
X_train

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # defines how many words are in our vocabulary (automatucally adds <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be
                                    pad_to_max_tokens=False # pad shorter sequences with zeros
                                    )

In [15]:
# Find the average number of tokens (words) in the training tweets
import numpy as np

np.round(np.sum([len(i.split()) for i in X_train]) / len(X_train))

15.0

In [16]:
# Setup text vectorization variables
max_vocab_length = 10000 #max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet can our model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the train data
text_vectorizer.adapt(X_train)

In [18]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(X_train)
print(f"Original text:\n{random_sentence}\
       \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
#TweetLikeItsSeptember11th2001 Those two buildings are on fire       

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[4359,  161,  116,   95,   22,   11,   42,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Num of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")


Num of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make our embedding we' re going to use a TensorFlow Embedding layer.

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example a value of 100 would mean token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [21]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

embedding

In [22]:
# Get a random sentence from the training set
random_sentence = random.choice(X_train)
print(f"Original text:\n {random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Tips so that finding the customers ego drought: dqSVYusY      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04367752,  0.00290947, -0.04566578, ..., -0.01385818,
          0.02374978, -0.00087406],
        [ 0.01470424,  0.04381484,  0.01295278, ...,  0.00053619,
          0.02796629, -0.01012247],
        [-0.00123105, -0.03251629, -0.02204552, ..., -0.01042131,
         -0.0136073 , -0.01887633],
        ...,
        [ 0.04189341, -0.03478963,  0.02570425, ...,  0.03345319,
          0.02820221,  0.02013233],
        [ 0.04189341, -0.03478963,  0.02570425, ...,  0.03345319,
          0.02820221,  0.02013233],
        [ 0.04189341, -0.03478963,  0.02570425, ...,  0.03345319,
          0.02820221,  0.02013233]]], dtype=float32)>

In [23]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.04367752,  0.00290947, -0.04566578,  0.02256176,  0.02636662,
        -0.01124039, -0.00065786,  0.04391403, -0.03129928, -0.04646579,
         0.01445813,  0.01151874, -0.00321422,  0.01691499, -0.02786224,
         0.0263876 , -0.00828861, -0.04336177, -0.01807914,  0.01141213,
         0.00658559,  0.03543338, -0.00240667, -0.01356167,  0.01956755,
         0.04827313,  0.03202974, -0.02978279, -0.0016476 ,  0.01365821,
        -0.0189845 , -0.04622942, -0.01249854,  0.03538349,  0.0326301 ,
         0.03160837,  0.01026088, -0.00070485,  0.04656999, -0.0031522 ,
         0.02373618, -0.01998464, -0.03556406, -0.04960089, -0.04098762,
        -0.01810107,  0.03881692,  0.01312071, -0.00256671, -0.01616474,
        -0.04399438, -0.02573199,  0.02223729, -0.04798779, -0.02257878,
        -0.03089967,  0.04231388, -0.04520656,  0.00809436, -0.01664648,
        -0.01380607, -0.04885979,  0.01863593, -0.01677047,  0.02291923,
  

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.
* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with tensorflow:
* Create a model
* Build a model
* Fit a model
* Evaluate our model

### Model 0: Getting a baseline

We'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

IT's common practice to use non-DL algorithms as a baseline because of their speed and then using DL to see if you can improve upon them.

In [24]:
# Create Naive Bayes model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers
    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
# Evaluate our baseline model
baseline_score = model_0.score(X_val, y_val)

In [26]:
baseline_score

0.7926509186351706

In [27]:
# Make predictions
baseline_preds = model_0.predict(X_val)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [66]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Create a function to output some classification evaluation metrics
def eval_metrics(y_true, y_pred):
  """
  Calculates accuracy, precision, recall and f1-score.

  Args:
    model: a ML model to make predictions on
    X_test: a tensor of test features
    y_test: a tensor of test labels
  
  Returns:
    a dict with all keys of all mentioned above metrics
    with values of those metrics
  """
  results = {}


  accuracy = accuracy_score(y_true=y_true,  
                            y_pred=y_pred)
  precision, recall, f1, _ = precision_recall_fscore_support(y_true=y_true,
                                                             y_pred = y_pred,
                                                             average="weighted")
  
  results["accuracy"] = accuracy
  results["precision"] = precision
  results["recall"] = recall
  results["f1"] = f1

  return results

In [67]:
# Check the evaluation metrics
baseline_results = eval_metrics(y_true=y_val,
                                y_pred=baseline_preds)

baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model

In [30]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback

# Create a directory to save tensorboard logs
SAVE_DIR = "model_logs"

In [55]:
# Build model with the Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [56]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [57]:
# Compile the model
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [58]:
# Fit the model
model_1_history = model_1.fit(x=X_train,
                              y=y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230416-090333
Epoch 1/5
215/215 [==============================] - 13s 54ms/step - loss: 0.5859 - accuracy: 0.7430 - val_loss: 0.5336 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4181 - accuracy: 0.8441 - val_loss: 0.4688 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3293 - accuracy: 0.8716 - val_loss: 0.4563 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2734 - accuracy: 0.8991 - val_loss: 0.4627 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2305 - accuracy: 0.9172 - val_loss: 0.4812 - val_accuracy: 0.7835


In [59]:
# Check the results
model_1.evaluate(X_val, y_val)

24/24 [==============================] - 0s 3ms/step - loss: 0.4812 - accuracy: 0.7835


[0.48116108775138855, 0.7834645509719849]

In [60]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(X_val)
model_1_pred_probs

24/24 [==============================] - 0s 4ms/step


array([[3.26517463e-01],
       [7.00781584e-01],
       [9.96478379e-01],
       [1.36815339e-01],
       [7.97762647e-02],
       [9.32446659e-01],
       [9.00663257e-01],
       [9.92139041e-01],
       [9.64510918e-01],
       [2.75467604e-01],
       [1.03167854e-01],
       [7.01371729e-01],
       [4.82834056e-02],
       [1.71713069e-01],
       [6.43744739e-03],
       [1.25400230e-01],
       [2.80832592e-02],
       [9.15437117e-02],
       [2.38600463e-01],
       [5.28415918e-01],
       [9.06372607e-01],
       [4.17299531e-02],
       [4.32931095e-01],
       [9.53286290e-02],
       [9.58945632e-01],
       [9.98447001e-01],
       [4.33216467e-02],
       [7.16198087e-02],
       [2.43141316e-02],
       [2.15944827e-01],
       [5.80829978e-01],
       [3.23988229e-01],
       [4.56927806e-01],
       [1.87828690e-01],
       [5.17964900e-01],
       [6.49735779e-02],
       [9.92742896e-01],
       [1.86357930e-01],
       [3.51321623e-02],
       [9.97872829e-01],


In [61]:
# Convert model prediction probabiliites to label format
model_1_pred_probs.shape

(762, 1)

In [62]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds.shape

TensorShape([762])

In [64]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [70]:
# Calculate our model_1 results
model_1_results = eval_metrics(y_true=y_val,
                               y_pred=model_1_preds)

model_1_results

{'accuracy': 0.7834645669291339,
 'precision': 0.7872123378365872,
 'recall': 0.7834645669291339,
 'f1': 0.7807800582578167}

## Visualizing learned embeddings

In [71]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [72]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [74]:
# Get the weight matrix of embedding layer
# (these are the numerical representations of each token in out training data which have been learned for 5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()
embed_weights[0].shape

(10000, 128)

Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize iter
To do so, TensorFlow has a handy tool called projector.

In [81]:
# Create embedding files (from tf's word embedding documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue # skip, 0 it's padding
  
  vec = embed_weights[0][index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")

out_v.close()
out_m.close()

In [82]:
# Download files from Colab to upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent neural networks (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.



### Model 2: LSTM

LSTM = long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)
```

In [108]:
# Create an LSTM model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(64, return_sequences=True)(x) # when you are stacking RNN cells together, you need to set return_sequences=True
# x = layers.Dropout(0.5)(x)
x = layers.LSTM(64)(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(64, activation="relu", kernel_regularizer="l2")(x)
# x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [109]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_12 (LSTM)              (None, 64)                49408     
                                                                 
 dense_17 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [110]:
# Compile the model
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [111]:
# Fit the model
model_2_history = model_2.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                    experiment_name="model_2_lstm")])

Saving TensorBoard log files to: model_logs/model_2_lstm/20230416-101633
Epoch 1/5
215/215 [==============================] - 14s 53ms/step - loss: 0.1009 - accuracy: 0.9612 - val_loss: 1.1561 - val_accuracy: 0.7428
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0351 - accuracy: 0.9834 - val_loss: 1.2700 - val_accuracy: 0.7520
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0354 - accuracy: 0.9837 - val_loss: 1.5052 - val_accuracy: 0.7362
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0360 - accuracy: 0.9838 - val_loss: 1.2720 - val_accuracy: 0.7441
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0311 - accuracy: 0.9847 - val_loss: 1.5523 - val_accuracy: 0.7585


In [112]:
# Make predictions
model_2_pred_probs = model_2.predict(X_val)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[7.0985977e-04],
       [7.2242898e-01],
       [9.9990785e-01],
       [9.6155358e-03],
       [7.9323603e-05],
       [9.9985862e-01],
       [9.9802279e-01],
       [9.9990308e-01],
       [9.9988091e-01],
       [3.6901779e-02]], dtype=float32)

In [113]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [114]:
# Calculate model 2 results
model_2_results = eval_metrics(y_true=y_val,
                               y_pred=model_2_preds)

model_2_results

{'accuracy': 0.7585301837270341,
 'precision': 0.7611862201281965,
 'recall': 0.7585301837270341,
 'f1': 0.7556142444864076}

### Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell, but has less parameters.

In [115]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences=True)(x) # if you want to stack recurrent layers on top of each other, you need to return_sequences=True
x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [118]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 15, 64)            37248     
                                                                 
 gru_1 (GRU)                 (None, 64)                24960     
                                                                 
 dense_18 (Dense)            (None, 64)                4160      
                                                       

In [116]:
# Compile the model
model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [117]:
# Fit the model
model_3_history = model_3.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230416-103431
Epoch 1/5
215/215 [==============================] - 19s 63ms/step - loss: 0.1107 - accuracy: 0.9629 - val_loss: 0.9828 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0417 - accuracy: 0.9806 - val_loss: 1.7895 - val_accuracy: 0.7467
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0365 - accuracy: 0.9832 - val_loss: 1.5450 - val_accuracy: 0.7441
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0331 - accuracy: 0.9839 - val_loss: 1.9318 - val_accuracy: 0.7415
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0327 - accuracy: 0.9845 - val_loss: 1.6704 - val_accuracy: 0.7598


In [119]:
# Make predictions
model_3_pred_probs = model_3.predict(X_val)
model_3_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[1.7650903e-04],
       [6.6469324e-01],
       [9.9997377e-01],
       [3.4497436e-02],
       [2.1452250e-05],
       [9.9994028e-01],
       [9.8031884e-01],
       [9.9997962e-01],
       [9.9997950e-01],
       [1.0233199e-02]], dtype=float32)

In [120]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [121]:
# Calcualte model 3 results
model_3_results = eval_metrics(y_true=y_val,
                               y_pred=model_3_preds)

In [122]:
model_3_results


{'accuracy': 0.7598425196850394,
 'precision': 0.7641410572035275,
 'recall': 0.7598425196850394,
 'f1': 0.7562134111850812}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right (just like you'd read an English sentence) however, a bidirectional RNN goes from left to right as well as from right to left.

In [123]:
# Build a bidirectional RNN in tensorflow
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [124]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                             

In [125]:
# Compile model
model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [126]:
# Fit the model
model_4_history = model_4.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230416-111636
Epoch 1/5
215/215 [==============================] - 30s 92ms/step - loss: 0.0748 - accuracy: 0.9755 - val_loss: 1.2311 - val_accuracy: 0.7441
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0344 - accuracy: 0.9823 - val_loss: 1.5575 - val_accuracy: 0.7480
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0337 - accuracy: 0.9826 - val_loss: 1.4549 - val_accuracy: 0.7375
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0288 - accuracy: 0.9851 - val_loss: 1.5858 - val_accuracy: 0.7546
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0281 - accuracy: 0.9848 - val_loss: 1.8128 - val_accuracy: 0.7638


In [127]:
# Make predictions
model_4_pred_probs = model_4.predict(X_val)
model_4_pred_probs[:10]

24/24 [==============================] - 4s 4ms/step


array([[1.7651978e-04],
       [9.5589209e-01],
       [9.9998486e-01],
       [2.3381744e-02],
       [1.0128436e-05],
       [9.9983668e-01],
       [9.9931884e-01],
       [9.9998796e-01],
       [9.9998415e-01],
       [5.5850949e-04]], dtype=float32)

In [128]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [129]:
# Calculate results
model_4_results = eval_metrics(y_true=y_val,
                               y_pred=model_4_preds)

model_4_results

{'accuracy': 0.7637795275590551,
 'precision': 0.7642156268927923,
 'recall': 0.7637795275590551,
 'f1': 0.7622584108804583}

## Convolutional Neural Networks for Text (and other types of sequences)

We've used CNN's for images but images are typically 2D (height x width). However, our text data is 1D.

Previously we've used Conv2D for our image data, but now we're going to use Conv1D.

The typical structure of a Conv1D model for sequences (in our case, text):

```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + pooling) -> Outputs (class probabilities)
```


### Model 5: 1D Convolutional neural network

In [134]:
# Test out our embedding layer, Conv1D layer and max pooling
from tensorflow.keras import layers

embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation="relu",
                        padding="valid")

conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to get the most important features

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape


(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [138]:
# Create a 1D Conv NN
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(32, 3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

# Create the model
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

In [139]:
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_4 (Conv1D)           (None, 13, 32)            12320     
                                                                 
 flatten_4 (Flatten)         (None, 416)               0         
                                                                 
 dense_23 (Dense)            (None, 1)                 417       
                                                    

In [140]:
# Compile the model
model_5.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [141]:
# Fit the model
model_5.fit(X_train,
            y_train,
            epochs=5,
            validation_data=(X_val, y_val),
            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                   experiment_name="model_5_Conv1D")])

Saving TensorBoard log files to: model_logs/model_5_Conv1D/20230416-113701
Epoch 1/5
215/215 [==============================] - 13s 48ms/step - loss: 0.1306 - accuracy: 0.9590 - val_loss: 0.8836 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0474 - accuracy: 0.9812 - val_loss: 1.0376 - val_accuracy: 0.7480
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0405 - accuracy: 0.9831 - val_loss: 1.1411 - val_accuracy: 0.7546
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0373 - accuracy: 0.9823 - val_loss: 1.1944 - val_accuracy: 0.7467
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0353 - accuracy: 0.9828 - val_loss: 1.2252 - val_accuracy: 0.7559
